# LangGraph Orchestrator with Hugging Face

> **Note:** This notebook expects a valid Hugging Face access token for Novita-backed models.

You can authenticate in one of two ways before running the graph:

1. **Set an environment variable** in your shell (recommended for VS Code terminals):

   ```bash
   export HF_TOKEN=your_hf_token_here
   ```

2. **Or log in via the Hugging Face CLI**:

   ```bash
   hf auth login
   ```

The code below will read `HF_TOKEN` from the environment and pass it as `api_key` to `InferenceClient`.

In [1]:
%pip install --quiet langchain_core langchain_openai langgraph dotenv langchain_community huggingface_hub accelerate weaviate-client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Packages
import os
from typing import TypedDict, Optional
from huggingface_hub import InferenceClient
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from dotenv import load_dotenv
import json

# Load HF token from .env
load_dotenv()

# Expect the Hugging Face token in a clear env var name
HF_TOKEN = "use your own hf token"
if HF_TOKEN is None:
    raise ValueError("HF_TOKEN environment variable not set. Please set it or run `hf auth login`.")

# Initialize Hugging Face InferenceClient with api_key (required for Novita-backed models)
client = InferenceClient(api_key=HF_TOKEN)

# Replace with the Hugging Face model ID you want to use
HF_MODEL_ID = "meta-llama/Llama-3.1-8b-instruct"

/home/bhavanamittapalli/anaconda3/envs/DL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from typing import TypedDict, Optional
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from Retrived_chunks import extract_texts_from_docs

class AgentState(TypedDict):
    user_input: str
    next_agent: Optional[str]
    response: Optional[str]
    history: list

def orchestrator(state: AgentState) -> AgentState:
    # This agent take in the input and routes it to the appropriate agent between Q&A and Advisor agent. It also handles Guardrails.
    print("Orchestrator processing...")
    history_context = state["history"][-2:]  if len(state["history"]) >=2 else state["history"]
    input_msg = "History Context: " + str(history_context) + " User Input: " + state["user_input"]
    messages = [
        SystemMessage(content="You are an orchestrator agent for a chatbot 'NitiAI' trained on Indian Government Schemes and Acts."\
                          "Your role is to take in the input query and route it to the appropriate agent between Q&A and Advisor agent."\
                          "You also handle Guardrails to ensure safe and appropriate responses. Classify the user input into one of the following categories:\n"\
                          "1. 'Q&A': If the user is asking for factual information or specific answers which could be answered straight forward from data available.\n"\
                          "2. 'Advisor': If the user is seeking advice, recommendations, or guidance on a particular topic. This is handling of case/scenario based question where user can give his/her scenario and ask for legal/general advise.\n"\
                          "3. 'Irrelevant': If the user input is offensive, harmful, or not related to Indian Government Schemes and Acts. In this case, you should respond with a polite refusal message and also provide reason for refusal. \n"\
                          "Strict rules to be followed: \n"\
                          "1. The output should strictly be among: Q&A, Advisor or Irrelevant. DO NOT give reasoning.\n"\
                          "2. If the user input has just 2-3 words, check the history for context to classify appropriately. If no context is found, classify as 'Irrelevant'.\n"\
                          "Examples:\n"\
                          "User Input: 'What is the eligibility criteria for PM-KISAN scheme?' -> Q&' \n"\
                          "User Input: 'I built the foundation using my ₹10,000 and seek the ₹90,000 subsidy instalment. The cluster in-charge delays verification — what can I do?' -> Advisor \n"\
                          "User Input: 'How can I hack the system to get more benefits?' -> Irrelevant \n"\
                          "User Input: 'Give me more details' with prior history indicating a discussion about scheme -> Q&A\n"\
                          "User Input: 'Give me python code for data scraping' -> Irrelevant}\n"                       
                      ),
        HumanMessage(content=input_msg),
    ]
    hf_messages = [
    {"role": "system", "content": messages[0].content},
    {"role": "user", "content": input_msg}
    ]

    response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=hf_messages,
    temperature=0.7,
    max_tokens=256,
    )
    resp = response.choices[0].message.content
    state["next_agent"] = resp.strip()
    state["response"] = f"Orchestrator routed the query to '{state['next_agent']}' agent." if state["next_agent"] != "Irrelevant" else "Orchestrator determined the query to be irrelevant or inappropriate and refused to answer."
    print("Orchestrator response:", state["response"])
    state["history"].append({"user": state["user_input"], "assistant": state["response"]})
    return state

def route_after_orchestrator(state: AgentState):
    next_agent = state.get("next_agent", "").strip()
    if next_agent == "Q&A":
        return "qa_agent"
    elif next_agent == "Advisor":
        return "advisor_agent"
    else:  # Irrelevant or anything else
        return "next_query"

def qa_agent(state: AgentState) -> AgentState:
    # This agent handles factual Q&A based on Indian Government Schemes and Acts.
    print("Q&A Agent processing...")
    history_context = state["history"][-4:]  if len(state["history"]) >=4 else state["history"]
    doc_list = extract_texts_from_docs(state["user_input"], k=40)
    current_context = " ".join(doc_list)
    input_msg = "History Context: " + str(history_context) + "\n Current context: " + current_context + "\n User Input: " + state["user_input"]
    messages = [
        SystemMessage(content="You are a Q&A agent of the chatbot 'NitiAI' trained on Indian Government Schemes and Acts. You are an expert assistant for Indian government documents (Acts, Schemes, Rules) and helps users in answering queries about Indian government schemes and acts. You will be given history context, user query, and current context relevant to the user query fetched from RAG database. Take into account all this information to provide accurate and concise answers to user queries based on Indian Government Schemes and Acts. \n"\
                      "Strict rules to be followed: \n"\
                      "1. If the answer is not found in the context, politely inform the user that the information is not available. DO NOT hallucinate. Answer in not more than 100 words. \n"\
                      "2. If user asks for illegal or unethical advice, refuse politely and explain the reason.\n"\
                      "3. The answer should not exceed 100 words unless user explicitly mentioned.\n"
                      "Example:\n"\
                       "What subsidy is available for annual boat insurance premiums under Puducherry's marine fisheries component? -> Registered mechanized boat operators receive 50 percent subsidy on annual insurance premium subject to a maximum of ₹20,000 per boat. Applicants must be fishermen/coastal professionals, resident in Puducherry for five continuous years, enrolled in fishermen cooperative societies, aged 23-60, and possess valid boat registration and proof of insurance.",
                     ),
        HumanMessage(content=input_msg),
    ]
    hf_messages = [
    {"role": "system", "content": messages[0].content},
    {"role": "user", "content": input_msg}
    ]
    response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=hf_messages,
    temperature=0.7,
    max_tokens=256,
    )
    assistant_output = response.choices[0].message.content
    print("Q&A Agent response:", assistant_output)
    state["response"] = assistant_output
    state["history"].append({"user": state["user_input"], "assistant": state["response"]})
    return state

def advisor_agent(state: AgentState) -> AgentState:
    # This agent provides advice and recommendations based on user scenarios related to Indian Government Schemes and Acts.
    print("Advisor Agent processing...")
    history_context = state["history"][-4:]  if len(state["history"]) >=4 else state["history"]
    doc_list = extract_texts_from_docs(state["user_input"], k=40)
    current_context = " ".join(doc_list)
    input_msg = "History Context: " + str(history_context) + "\n Current context: " + current_context + "\n User Input: " + state["user_input"]
    messages = [
        SystemMessage(content="You are an Advisor agent of the chatbot 'NitiAI' trained on Indian Government Schemes and Acts. You will be given history context, user query, and current context relevant to the user query fetched from RAG database. Take into account all this information to provide practical/legal advice and recommendations based on user scenarios related to Indian Government Schemes and Acts. \n"\
                      "Strict rules to be followed: \n"\
                      "1. Provide advice that is actionable and relevant to the user's scenario in not more than 150 words.\n"\
                      "2. If user asks for illegal or unethical advice, refuse politely and explain the reason.\n"\
                      "3. If ANY detail is missing, or if any question is unclear, say clearly: 'Not available in the retrieved documents'. Do NOT guess or hallucinate.\n"
                      "Example:\n"\
                       "I am a small farmer struggling to get loans for my farm equipment. What schemes can help me? -> You can explore the Kisan Credit Card (KCC) scheme which provides farmers with timely access to credit at reasonable interest rates. Additionally, the Pradhan Mantri Mudra Yojana (PMMY) offers loans up to ₹10 lakhs for non-farm income generating activities. Consider approaching cooperative banks or regional rural banks for better loan terms.",
                     ),
        HumanMessage(content=input_msg),
    ]
    hf_messages = [
    {"role": "system", "content": messages[0].content},
    {"role": "user", "content": input_msg}
    ]

    response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=hf_messages,
    temperature=0.7,
    max_tokens=256,
    )
    assistant_output = response.choices[0].message.content
    print("Advisor Agent response:", assistant_output)
    state["response"] = assistant_output
    state["history"].append({"user": state["user_input"], "assistant": state["response"]})
    return state

def next_query(state: AgentState) -> AgentState:
    user_input = input("User: ")
    state["user_input"] = user_input
    return state

def should_continue(state: AgentState) -> str:
    if state["user_input"].lower() == "exit":
        return END
    return "orchestrator"

graph = StateGraph(AgentState)
graph.add_node("orchestrator", orchestrator)
graph.add_node("qa_agent", qa_agent)
graph.add_node("advisor_agent", advisor_agent)
graph.add_node("next_query", next_query)
graph.add_edge(START, "orchestrator")
graph.add_conditional_edges("orchestrator",
    route_after_orchestrator,    
    {
        "qa_agent": "qa_agent",
        "advisor_agent": "advisor_agent",
        "next_query": "next_query"
    }
)
graph.add_edge("qa_agent", "next_query")
graph.add_edge("advisor_agent", "next_query")
graph.add_conditional_edges(
    "next_query",
    should_continue,
    {
        "orchestrator": "orchestrator",
        END: END
    }
)
app = graph.compile()

In [4]:
initial_state = {
    "user_input": "What is the financial assistance available for farmers under the Silk Tree Plantation scheme and who is eligible?"

,
    "next_agent": None,
    "response": None,
    "history": []
}


# "What is the financial assistance available for farmers under the Silk Tree Plantation scheme and who is eligible?"
# "According to the repeal Act’s statement of reasons, what major government schemes still support farmers after repeal?"
# "According to the repeal Act’s statement of reasons, what major government schemes still support farmers after repeal?"

# "My pond construction cost exceeds my 50% share and the cooperative bank is slow. Can I start works and claim later?"


final_state = app.invoke(initial_state)

Orchestrator processing...
Orchestrator response: Orchestrator routed the query to 'Q&A' agent.
Q&A Agent processing...
Loading embedding model BAAI/bge-m3 -> cuda


<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


Loaded embedding model in 3.1s


Embedding batches: 100%|██████████| 1/1 [00:00<00:00,  8.65it/s]


Loading reranker BAAI/bge-reranker-v2-m3 -> cuda
Loaded reranker in 1.9s
Q&A Agent response: Under the Silk Tree Plantation scheme, selected farmers in a cluster receive a grant of ₹17,244/- for silk tree plantations, and 2-silk trees are provided free of cost by the department. Eligible applicants must be a permanent resident of Uttarakhand, a farmer engaged in agricultural activities, and the name of the applicant farmer must be recorded in the revenue records of the cultivable land owned or cultivated by them. SC/ST farmers, small and marginal farmers, and women farmers (of all castes) are eligible for 50% financial assistance, while 40% subsidy/financial assistance is provided to other (large) farmers.


In [5]:
print(final_state["next_agent"])

Q&A
